# Lab 3 - Optymalizacja hiperparametrów sieci neuronowej

Optymalizacja hiperparametrów to najważniejszy etap podczas trenowania sieci neuronowej. Hiperparametry, takie jak współczynnik uczenia (learning rate), liczba neuronów w warstwach, wielkość grupy (batch size) czy współczynnik porzucania (dropout), nie są bezpośrednio optymalizowane podczas treningu modelu, ale mają znaczący wpływ na jego wydajność. Dobrze dobrane hiperparametry mogą znacząco poprawić dokładność, szybkość uczenia i zdolność generalizacji modelu. Proces ich ręcznego doboru bywa jednak czasochłonny i opiera się często na intuicji lub metodzie prób i błędów.

Z uwagi na wymienione okoliczności, powszechnie wykorzystuje się automatyczne narzędzia do optymalizacji hiperparametrów, takie jak **Optuna**. Jest to biblioteka otwartoźródłowa, przeznaczona do optymalizacji funkcji kosztu, stworzona specjalnie z myślą o prostocie i wydajności. Optuna wykorzystuje złożone techniki eksploracji przestrzeni parametrów, m.in. sekwencyjną optymalizację Bayesowską, by znaleźć optymalne ustawienia hiperparametrów przy minimalnej liczbie prób. Dzięki temu potrafi szybciej znaleźć lepsze rozwiązania niż metody takie jak deterministyczne lub stochastyczne przeszukiwanie siatki.

Biblioteka Optuna dobrze integruje się z popularnymi bibliotekami do uczenia maszynowego, takimi jak TensorFlow, PyTorch, czy scikit-learn. Umożliwia definiowanie funkcji celu jako zwykłych funkcji języka Python, w których można dowolnie próbować różnych architektur modelu, parametrów optymalizatora, czy strategii regularizacji. Co więcej, Optuna oferuje przyjazne narzędzia do wizualizacji postępów i wyników optymalizacji, co ułatwia analizę procesu i interpretację wyników.

## Optymalizacja nieliniowej funkcji jednej zmiennej

$f(x) = (x - 2)^2 + 1$

Cel: minimalizacja parametru $x$

In [ ]:
!pip install optuna

In [4]:
from typing import Any

import optuna

In [6]:
def f(x: float) -> float:
  return (x - 2)**2 + 1

In [7]:
def objective(trial: Any) -> float:
    x = trial.suggest_float("x", -10, 10)
    return f(x)

Uruchomienie optymalizacji

In [ ]:
study = optuna.create_study(direction="minimize")

In [ ]:
study.optimize(objective, n_trials=100)

Wyniki

In [ ]:
print(f" Dla x = {study.best_params['x']}, minimalna warość funkcji wynosi: {study.best_value}")

## Optymalizacja funkcji wielu zmiennych

$f(x, y) = (x - 3)^2 + (y + 1)^2 + 5$

In [11]:
def f(x: float, y: float) -> float:
  return (x - 3)**2 + (y + 1)**2 + 5

In [12]:
def objective(trial: Any) -> float:
  x = trial.suggest_float("x", -10, 10)
  y = trial.suggest_float("y", -10, 10)

  return f(x, y)

Uruchomienie optymalizacji

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)

In [ ]:
print(f" Dla x = {study.best_params['x']}, y = {study.best_params['y']}, minimalna warość funkcji wynosi: {study.best_value}")

## Optymalizacja hiperparametrów sieci neuronowej

In [5]:
from typing import Any

import numpy as np
import optuna
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

Wczytanie danych

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

Funkcja celu: maksymalizacja dokładności zbalansowanej

In [10]:
def objective(trial: Any) -> float:
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))

    n_units = trial.suggest_int("n_units", 32, 128)
    model.add(Dense(n_units, activation="relu"))

    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    model.add(Dropout(dropout_rate))

    model.add(Dense(10, activation="softmax"))

    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    model.compile(optimizer=optimizer,
                  loss="categorical_crossentropy",
                  metrics=["categorical_accuracy"])

    model.fit(x_train, y_train,
              batch_size=trial.suggest_categorical("batch_size", [32, 64, 128]),
              epochs=5,
              verbose=2,
    )

    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    return accuracy

Optymalizacja hiperparametrów:
- n_units: liczba neuronów w warstwie ukrytej,
- dropout_rate: prawdopodobieństwo porzucenia neuronu,
- lr: współczynnik uczenia,
- batch size: rozmiar grupy obiektów.

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

Najlepsze hiperparametry

In [ ]:
for key, val in study.best_params.items():
    print(f"{key}: {val}")

In [ ]:
print(f"Najlepszy wynik walidacji: {study.best_value:.4f}")

## Zadania

1. Zaimplementować funkcję nieliniową $f(x) = (x = 5)^2 + 3$ i wykorzystać bibliotekę Optuna do znalezienia wartości x minimalizującej tę funkcję.
2. Zdefiniować funkcję dwóch zmiennych $f(x, y) = sin(x) + cos(y) + 0.1(x^2 + y^2)$. Użyć Optuny, aby znaleźć wartości x i y, które minimalizują tę funkcję.
3. Zaprojektować funkcję celu opartą na kombinacji trzech zmiennych: $f(x, y, x) = a^2 + b^2 + c^2 - ab + sin(c)$, a następnie znaleźć minimalną wartość tej funkcji za pomocą Optuny.
4. Zoptymalizować parametry funkcji $f(x, y) = e ^ {x ^ {2} + y ^ {2}} - 2x + 3y$, gdzie $x, y \in [-2, 2]$. Porównać wynik z metodą deterministycznego przeszukiwania siatki.
5. Zbudować splotową sieć neuronową w TensorFlow do klasyfikacji obrazów ze zbioru CIFAR-10. Ograniczyć podzbiór treningowy do 15000 przykładów, a liczbę epok do maks. 15. Należy także zadbać o zastosowanie podstawowych technik poprawiających uogólnianie (Dropout, BatchNormalization). Ocenić dokładność na podzbiorze walidacyjnym.
6. Wykorzystać bibliotekę Optuna do optymalizacji struktury sieci z zadania 5. W tym celu należy dobrać:
- liczbę filtrów w każdej warstwie splotowej (zwykle 16, 32, 64, ...),
- liczbę warstw splotowych (zwykle 1-9),
- rozmiar jądra splotu (zwykle 3x3, 5x5, ...),
- funkcje aktywacji.
7. Rozszerzyć eksperyment z zadania 6 o optymalizację strategii treningu. Wykorzystać Optunę do dobrania:
- learning rate (1e-5 do 1e-2, z log=True),
- optymalizatora (adam, sgd, rmsprop),
- współczynnika Dropout (0.1–0.5),
- wielkości grupy (16, 32, 64, ...).
8. Użyć zbioru danych Adult (dostępny np. w sklearn.datasets.fetch_openml) do klasyfikacji (czy osoba zarabia >50k). Zbudować potok, który:
- automatycznie koduje dane kategorialne,
- skaluje dane numeryczne,
- dobiera liczbę warstw ukrytych i neuronów dla każdej warstwy (Optuna),
- testuje różne funkcje aktywacji (relu, selu, elu, tanh).
